In [1]:
from pynq import Overlay
ol = Overlay("matmult_stream_2022.bit")

In [2]:
ol?

In [3]:
data_channel = ol.axi_dma_0

# Create Data

In [4]:
import numpy as np

input_A = np.random.randint(1, 21, size=(8, 8))
input_B = np.random.randint(1, 21, size=(8, 8))

# PS Implementation

In [5]:
output_C_ps = np.dot(input_A, input_B)
output_C_ps

array([[ 894, 1167,  835,  715,  684,  696,  800,  662],
       [ 925,  968,  649,  914,  707,  902,  894,  606],
       [1301, 1601,  956,  940, 1115, 1225, 1158,  832],
       [ 890,  883,  699,  779,  763,  955,  915,  633],
       [1358, 1594,  992, 1240, 1190, 1202, 1107,  944],
       [1073, 1141,  754,  687,  777,  809,  850,  672],
       [1179, 1211,  898,  877,  937, 1190, 1165,  787],
       [1189, 1408,  881, 1032,  981, 1056,  987,  892]])

# PL Implementation

In [6]:
from pynq import allocate

In [7]:
input_buffer = allocate((16,8),np.single)
output_buffer = allocate((8,8),np.single)

In [8]:
send_channel = data_channel.sendchannel
recv_channel = data_channel.recvchannel

In [9]:
matrix_16x8 = np.vstack((input_A, input_B))
matrix_16x8

array([[ 1,  2, 17,  7, 20, 17,  7,  9],
       [ 4, 17,  2, 17,  1, 17, 10, 17],
       [16, 15, 17, 13, 20,  2, 16, 20],
       [12, 13, 18, 16,  7,  5,  3,  7],
       [16,  8, 19, 15, 11, 19, 14, 20],
       [ 7,  9, 19,  6, 12, 12,  5, 16],
       [15, 19, 19, 18, 14,  2,  5, 13],
       [18, 15,  3, 14, 18, 14, 10, 17],
       [11, 11,  2, 14, 18, 11,  1, 13],
       [13,  6,  3,  2, 10, 16, 18,  8],
       [14, 12, 10,  6, 13, 13, 14,  7],
       [ 4,  5, 15, 20,  2, 16, 16,  5],
       [10, 20, 16,  2,  4,  5, 10, 10],
       [12, 13,  9, 18, 11,  6,  7, 14],
       [ 1, 20,  1, 13, 18, 13,  7,  2],
       [20, 16,  8,  2,  2,  3,  5,  3]])

In [10]:
np.copyto(input_buffer, matrix_16x8)

In [11]:
input_buffer?

In [12]:
input_A

array([[ 1,  2, 17,  7, 20, 17,  7,  9],
       [ 4, 17,  2, 17,  1, 17, 10, 17],
       [16, 15, 17, 13, 20,  2, 16, 20],
       [12, 13, 18, 16,  7,  5,  3,  7],
       [16,  8, 19, 15, 11, 19, 14, 20],
       [ 7,  9, 19,  6, 12, 12,  5, 16],
       [15, 19, 19, 18, 14,  2,  5, 13],
       [18, 15,  3, 14, 18, 14, 10, 17]])

In [13]:
input_B

array([[11, 11,  2, 14, 18, 11,  1, 13],
       [13,  6,  3,  2, 10, 16, 18,  8],
       [14, 12, 10,  6, 13, 13, 14,  7],
       [ 4,  5, 15, 20,  2, 16, 16,  5],
       [10, 20, 16,  2,  4,  5, 10, 10],
       [12, 13,  9, 18, 11,  6,  7, 14],
       [ 1, 20,  1, 13, 18, 13,  7,  2],
       [20, 16,  8,  2,  2,  3,  5,  3]])

In [14]:
send_channel.transfer(input_buffer)
recv_channel.transfer(output_buffer)
send_channel.wait()
recv_channel.wait()

In [15]:
output_C_pl = np.array([[0]*8]*8, dtype = np.single)
np.copyto(output_C_pl, output_buffer)

# Comparison between PS and PL Output

In [16]:
print(output_C_ps)
print(output_C_pl)

[[ 894 1167  835  715  684  696  800  662]
 [ 925  968  649  914  707  902  894  606]
 [1301 1601  956  940 1115 1225 1158  832]
 [ 890  883  699  779  763  955  915  633]
 [1358 1594  992 1240 1190 1202 1107  944]
 [1073 1141  754  687  777  809  850  672]
 [1179 1211  898  877  937 1190 1165  787]
 [1189 1408  881 1032  981 1056  987  892]]
[[  894.  1167.   835.   715.   684.   696.   800.   662.]
 [  925.   968.   649.   914.   707.   902.   894.   606.]
 [ 1301.  1601.   956.   940.  1115.  1225.  1158.   832.]
 [  890.   883.   699.   779.   763.   955.   915.   633.]
 [ 1358.  1594.   992.  1240.  1190.  1202.  1107.   944.]
 [ 1073.  1141.   754.   687.   777.   809.   850.   672.]
 [ 1179.  1211.   898.   877.   937.  1190.  1165.   787.]
 [ 1189.  1408.   881.  1032.   981.  1056.   987.   892.]]
